### Config file generator
This simple notebook provides a tool for generating map2loop config hjson files, one of the more annoying parts of the process.
   
1. It first gets the project name and destination projection system (as EPSG number)   
   
2. It then asks for the paths to the different shape files (for WFS you will have to edit this manually afterwards)   
   
3. Finally for each input file, you have to define which fields to use, and which keywords to look for in some of these fields   
     
4. Finally, make a copy of the <a href="Example 3 - Local Source Data.ipynb" target="_blank">Generic local file notebook</a> and on line 14 of the second cell change **metadata='path_to_your_new_hjson_file'**) 


In [50]:
#!conda install ipyfilechooser
from ipyfilechooser import FileChooser
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
import os
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
pd.options.display.max_columns = None
style = {'description_width': 'initial'}
c_l= {
#Orientations
  "d": "dip",                  #field that contains dip information
  "dd": "azimuth",             #field that contains dip direction information
  "sf": 'codedescpt',             #field that contains information on type of structure
  "bedding": 'bedding',            #text to search for in field defined by sf code to show that this is a bedding measurement
  "otype": 'dip direction',            #flag to determine measurement convention (currently 'strike' or 'dip direction')
  "bo": "deform",             #field that contains type of foliation
  "btype": 'BEOI',            #text to search for in field defined by bo code to show that this is an overturned bedding measurement
#stratigraphy
  "g": 'sub_provin',               #field that contains coarser stratigraphic coding
  "g2": 'province',              #field that contains alternate coarser stratigraphic coding if 'g' is blank
  "c": 'unit_name',                 #field that contains finer stratigraphic coding
  "ds": 'deposition',           #field that contains information about lithology
  "u": 'nsw_code',             #field that contains alternate stratigraphic coding (not used??)
  "r1": 'class',           #field that contains  extra lithology information
  "r2": 'igneous_ty',           #field that contains even more lithology information
  "sill": 'sill',              #text to search for in field defined by ds code to show that this is a sill
  "intrusive": 'intrusive',    #text to search for in field defined by r1 code to show that this is an intrusion
  "volcanic": 'volcanic',      #text to search for in field defined by ds code to show that this is an volcanic (not intrusion)
#mineral deposits
  "msc": 'SITE_CODE',          #field that contains site code of deposit
  "msn": 'SHORT_NAME',         #field that contains short name of deposit
  "mst": 'SITE_TYPE_',         #field that contains site type of deposit
  "mtc": 'TARGET_COM',         #field that contains target commodity of deposit
  "mscm": 'SITE_COMMO',        #field that contains site commodity of deposit
  "mcom": 'COMMODITY_',        #field that contains commodity group of deposit
  "minf": 'Infrastructure',    #text to search for in field defined by mst code that shows site to ignore
#timing
  "min": 'top_end_ag',         #field that contains minimum age of unit defined by ccode
  "max": 'base_start',         #field that contains maximum age of unit defined by ccode
#faults and folds
  "f": 'boundaryty',              #field that contains information on type of structure
  "fault": 'Fault',            #text to search for in field defined by f code to show that this is a fault
  "ff": 'codedescpt',  # field that contains information on type of structure
  "fold": 'cline',  #text to search for in field defined by ff code to show that this is a fold axial trace
  "fdip": 'dip',               # field for numeric fault dip value
  "fdipnull": '0',         # text to search for in field defined by fdip to show that this has no known dip
  "fdipdir": 'faultdipdi',        # field for text fault dip direction value 
  "fdipdir_flag": 'alpha',        # flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast    
  "fdipest": 'faultdipan',        # field for text fault dip estimate value
  "fdipest_vals": 'Moderate,Listric,Steep,Vertical',        # text to search for in field defined by fdipest to give fault dip estimate in increasing steepness
  "n": 'name',                 #field that contains information on name of fault (not used??)
  "t": 'codedescpt',                 #field that contains information on type of fold
  "syn": 'Syncline',           #text to search for in field defined by t to show that this is a syncline
#ids
  "o": 'unique_id',             #field that contains unique id of geometry object
  "gi": 'objectid',            #field that contains unique id of structure point
  "deposit_dist":500           # dist between mindep and contact to be considered "on contact"
}

In [2]:
test_data_name_choice=widgets.Text(
    placeholder='Test',
    value='Test',
    description='Project Name:',
    disabled=False,
    style=style,
    tooltip='Name of directory to hold config file (relative path)'
)
display(test_data_name_choice)

epsg=pd.read_csv('source_data/epsg.csv')

crs_choice=widgets.Dropdown(
    options=epsg['epsg'],
    description='CRS EPSG:',
    disabled=False
)
display(crs_choice)


Text(value='Test', description='Project Name:', placeholder='Test', style=DescriptionStyle(description_width='…

Dropdown(description='CRS EPSG:', options=('28348: GDA94 // MGA zone 48', '28349: GDA94 // MGA zone 49', '2835…

In [52]:
test_data_name=test_data_name_choice.value
epsg_code=crs_choice.value.split(":")
dst_crs='epsg:'+epsg_code[0]
printmd('**'+test_data_name+", "+dst_crs+'**')

**outtest2, epsg:27700**

In [53]:
minx_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min x:',
    disabled=False,
    style=style
)
display(minx_choice)

maxx_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max x:',
    disabled=False,
    style=style
)
display(maxx_choice)

miny_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min y:',
    disabled=False,
    style=style
)
display(miny_choice)

maxy_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max y:',
    disabled=False,
    style=style
)
display(maxy_choice)

minz_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min z:',
    disabled=False,
    style=style
)
display(minz_choice)

maxz_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max z:',
    disabled=False,
    style=style
)
display(maxz_choice)


Text(value='0', description='min x:', placeholder='0', style=DescriptionStyle(description_width='initial'))

Text(value='1', description='max x:', placeholder='1', style=DescriptionStyle(description_width='initial'))

Text(value='0', description='min y:', placeholder='0', style=DescriptionStyle(description_width='initial'))

Text(value='1', description='max y:', placeholder='1', style=DescriptionStyle(description_width='initial'))

Text(value='0', description='min z:', placeholder='0', style=DescriptionStyle(description_width='initial'))

Text(value='1', description='max z:', placeholder='1', style=DescriptionStyle(description_width='initial'))

In [54]:
minx=minx_choice.value
maxx=maxx_choice.value
miny=miny_choice.value
maxy=maxy_choice.value
minz=minz_choice.value
maxz=maxz_choice.value


structure_file_m = FileChooser('../')
structure_file_m.title = '<b>Choose Bedding Orientation point file</b>'
display(structure_file_m)

fault_file_m = FileChooser('../')
fault_file_m.title = '<b>Choose Fault polyline file</b>'
display(fault_file_m)

fold_file_m = FileChooser('../')
fold_file_m.title = '<b>Choose Fold polyline file</b>'
display(fold_file_m)

geology_file_m = FileChooser('../')
geology_file_m.title = '<b>Choose Geology polygon file</b>'
display(geology_file_m)

mindep_file_m = FileChooser('../')
mindep_file_m.title = '<b>Choose Mineral Deposit point file (or use null_mindeps.shp from source_data directory)</b>'
#mindep_file_m.default_path = '../source_data/'
mindep_file_m.default_filename = 'null_mindeps.shp'
display(mindep_file_m)



FileChooser(path='../', filename='', title='HTML(value='<b>Choose Bedding Orientation point file</b>')', show_…

FileChooser(path='../', filename='', title='HTML(value='<b>Choose Fault polyline file</b>')', show_hidden='Fal…

FileChooser(path='../', filename='', title='HTML(value='<b>Choose Fold polyline file</b>')', show_hidden='Fals…

FileChooser(path='../', filename='', title='HTML(value='<b>Choose Geology polygon file</b>')', show_hidden='Fa…

FileChooser(path='../', filename='null_mindeps.shp', title='HTML(value='<b>Choose Mineral Deposit point file (…

In [55]:
if(str(structure_file_m.selected_path) =='None'):
    structure_file=''
else:
    structure_file=structure_file_m.selected_path.replace('\\','/')+'/'+structure_file_m.selected_filename
if(str(fault_file_m.selected_path) =='None'):
    fault_file=''
else:
    fault_file=fault_file_m.selected_path.replace('\\','/')+'/'+fault_file_m.selected_filename
if(str(fold_file_m.selected_path) =='None'):
    fold_file=''
else:
    fold_file=fold_file_m.selected_path.replace('\\','/')+'/'+fold_file_m.selected_filename
if(str(geology_file_m.selected_path) =='None'):
    geology_file=''
else:
    geology_file=geology_file_m.selected_path.replace('\\','/')+'/'+geology_file_m.selected_filename
if(str(mindep_file_m.selected_path) =='None'):
    mindep_file=''
else:
    mindep_file=mindep_file_m.selected_path.replace('\\','/')+'/'+mindep_file_m.selected_filename

display(structure_file,fault_file,fold_file,geology_file,mindep_file)

'C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Strike_dip.shp'

'C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Faults.shp'

'C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Faults.shp'

'C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Bedrock.shp'

'C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/null_mindeps.shp'

In [56]:
if(not structure_file==''):
    structure=gpd.read_file(structure_file)
    display(structure.head())
    
    cols=['No_col']
    for sc in structure.columns:
        cols.append(sc)
        
    printmd('**Based on file: '+structure_file+'**')
    printmd('\n**Field that contains unique id of structure point**')
    gi_choice=widgets.Dropdown(
        options=cols,
        description='Unique ID',
        disabled=False

    )
    display(gi_choice)

    printmd('**Field that contains information on type of structure (e.g. bedding, cleavage etc.)**')

    sf_choice=widgets.Dropdown(
        options=cols,
        description='Structure',
        disabled=False
    )
    display(sf_choice)

    printmd('**Text to search for in field defined by type of structure code to show that this is a bedding measurement**')

    bedding_choice=widgets.Text(
        placeholder='Bed',
        value='Bed',
        description='Bedding text',
        disabled=False
    )
    display(bedding_choice)

    printmd('**Field that contains dip information**')

    d_choice=widgets.Dropdown(
        options=cols,
        description='Dip',
        disabled=False
    )
    display(d_choice)

    printmd('**Field that contains dip direction or strike information**')

    dd_choice=widgets.Dropdown(
        options=cols,
        description='Dip Direction',
        disabled=False
    )
    display(dd_choice)

    printmd('**Flag to determine measurement convention**')

    otype_choice=widgets.Dropdown(
        options=('dip direction','strike'),
        description='Convention',
        disabled=False
    )
    display(otype_choice)

    printmd('**Field that contains polarity of foliation (upward facing or not)**')

    bo_choice=widgets.Dropdown(
        options=cols,
        description='Polarity',
        disabled=False
    )
    display(bo_choice)

    printmd('**Text to search for in field defined by Polarity code to show that this is an overturned bedding measurement**')

    btype_choice=widgets.Text(
        placeholder='Blah',
        value='Blah',
        description='Pol. text',
        disabled=False
    )
    display(btype_choice)



,OBJECTID,AZIMUTH,DIP,X,Y,FEATURE,MINERAL,SYMBOL,NEWBGSVS2,geometry
0,18906,0.0,0,401553.0,374480.0,Strata_Horizontal,No Mineral,S_HS,Horizontal strata,POINT Z (401553.000 374480.000 0.000)
1,18907,0.0,0,401879.0,372018.0,Strata_Horizontal,No Mineral,S_HS,Horizontal strata,POINT Z (401879.000 372018.000 0.000)
2,18908,0.0,0,404052.0,373527.0,Strata_Horizontal,No Mineral,S_HS,Horizontal strata,POINT Z (404052.000 373527.000 0.000)
3,18910,0.0,0,401567.0,372615.0,Strata_Horizontal,No Mineral,S_HS,Horizontal strata,POINT Z (401567.000 372615.000 0.000)
4,18911,0.0,0,408178.0,367292.0,Strata_Horizontal,No Mineral,S_HS,Horizontal strata,POINT Z (408178.000 367292.000 0.000)


**Based on file: C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Strike_dip.shp**


**Field that contains unique id of structure point**

Dropdown(description='Unique ID', options=('No_col', 'OBJECTID', 'AZIMUTH', 'DIP', 'X', 'Y', 'FEATURE', 'MINER…

**Field that contains information on type of structure (e.g. bedding, cleavage etc.)**

...

Dropdown(description='Convention', options=('dip direction', 'strike'), value='dip direction')

**Field that contains polarity of foliation (upward facing or not)**

Dropdown(description='Polarity', options=('No_col', 'OBJECTID', 'AZIMUTH', 'DIP', 'X', 'Y', 'FEATURE', 'MINERA…

**Text to search for in field defined by Polarity code to show that this is an overturned bedding measurement**

Text(value='Blah', description='Pol. text', placeholder='Blah')

In [57]:
c_l['gi']=gi_choice.value
c_l['sf']=sf_choice.value
c_l['bedding']=bedding_choice.value
c_l['d']=d_choice.value
c_l['dd']=dd_choice.value
c_l['otype']=otype_choice.value
c_l['bo']=bo_choice.value
c_l['btype']=btype_choice.value

if(not fault_file==''):
    fault=gpd.read_file(fault_file)
    display(fault.head())
    printmd('**Based on file: '+fault_file+'**')

    cols=['No_col']
    for sc in fault.columns:
        cols.append(sc)
        
    printmd('\n**Field that contains information on type of structure (fault, fold axial trace etc.)**')


    f_choice=widgets.Dropdown(
        options=cols,
        description='Fault field:',
        disabled=False
    )
    display(f_choice)

    printmd('**Text to search for in field defined by Fault field code to show that this is a fault**')

    fault_choice=widgets.Text(
        placeholder='Fault',
        value='Fault',
        description='Fault text:',
        disabled=False
    )
    display(fault_choice)

    printmd('**Field for numeric fault dip value**')

    fdip_choice=widgets.Dropdown(
        options=cols,
        description='Dip:',
        disabled=False
    )
    display(fdip_choice)

    printmd('**Text to search for in field defined by Dip to show that this has no known dip**')

    fdipnull_choice=widgets.Text(
        placeholder='0',
        value='0',
        description='NULL dip:',
        disabled=False
    )
    display(fdipnull_choice)

    printmd('**Field for text fault dip direction value (southeast etc.)**')

    fdipdir_choice=widgets.Dropdown(
        options=cols,
        description='Dip Direction:',
        disabled=False
    )
    display(fdipdir_choice)

    printmd('**Flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast**')

    fdipdir_flag_choice=widgets.Dropdown(
        options=('alpha','num'),
        description='Dip Convention:',
        disabled=False,
        style=style
    )
    display(fdipdir_flag_choice)

    printmd('**Field for text fault dip estimate value (Moderate, Steep etc.)**')

    fdipest_choice=widgets.Dropdown(
        options=cols,
        description='Dip estimate:',
        disabled=False
    )
    display(fdipest_choice)

,CATEGORY,FEATURE,FEATURE_D,HWALL_ROSE,geometry
0,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH_EAST,"LINESTRING (415372.000 355736.000, 415462.000 ..."
1,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH,"LINESTRING (419427.000 359339.000, 419546.000 ..."
2,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH_WEST,"LINESTRING (421657.000 359077.000, 421802.000 ..."
3,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH,"LINESTRING (422552.000 370743.000, 422671.000 ..."
4,FAULT,Fault_Inf_HW,"Fault, inferred",EAST,"LINESTRING (409016.000 364784.000, 409039.000 ..."


**Based on file: C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Faults.shp**


**Field that contains information on type of structure (fault, fold axial trace etc.)**

Dropdown(description='Fault field:', options=('No_col', 'CATEGORY', 'FEATURE', 'FEATURE_D', 'HWALL_ROSE', 'geo…

**Text to search for in field defined by Fault field code to show that this is a fault**

...

Dropdown(description='Dip Direction:', options=('No_col', 'CATEGORY', 'FEATURE', 'FEATURE_D', 'HWALL_ROSE', 'g…

**Flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast**

Dropdown(description='Dip Convention:', options=('alpha', 'num'), style=DescriptionStyle(description_width='in…

**Field for text fault dip estimate value (Moderate, Steep etc.)**

Dropdown(description='Dip estimate:', options=('No_col', 'CATEGORY', 'FEATURE', 'FEATURE_D', 'HWALL_ROSE', 'ge…

In [58]:
printmd('**Text to search for in field defined by fdipest to give fault dip estimate in increasing steepness**')

if(fdipest_choice.value=='No_col'):
    c_l['fdipest_vals']='No_col'
else:
    fdipest_vals_choice=widgets.Text(
        placeholder='Sorted dip order (shallow to steep)',
        value=str(fault[fdipest_choice.value].unique()).replace("[","").replace("]","").replace(" ",","),
        description='Text Dips:',
        disabled=False
    )
    #display(fdipest_vals)
    c_l['fdipest_vals']=fdipest_vals_choice.value


    
printmd('**Field that contains information on name of fault (not used??)**')

n_choice=widgets.Dropdown(
    options=cols,
    description='Name:',
    disabled=False
)
display(n_choice)

**Text to search for in field defined by fdipest to give fault dip estimate in increasing steepness**

**Field that contains information on name of fault (not used??)**

Dropdown(description='Name:', options=('No_col', 'CATEGORY', 'FEATURE', 'FEATURE_D', 'HWALL_ROSE', 'geometry')…

In [59]:
c_l['f']=f_choice.value
c_l['fault']=fault_choice.value
c_l['fdip']=fdip_choice.value
c_l['fdipnull']=fdipnull_choice.value
c_l['fdipdir']=fdipdir_choice.value
c_l['fdipdir_flag']=fdipdir_flag_choice.value
c_l['fdipest']=fdipest_choice.value
c_l['n']=n_choice.value

if(not fold_file==''):
    fold=gpd.read_file(fold_file)
    display(fold.head())
    cols=['No_col']
    for sc in fold.columns:
        cols.append(sc)
        
    printmd('**Based on file:'+' '+fold_file+'**')

    printmd('\n**Field that contains information on type of structure (fold, fault etc.)**')

    ff_choice=widgets.Dropdown(
        options=cols,
        description='Fold field:',
        disabled=False
    )
    display(ff_choice)

    printmd('**Text to search for in field defined by Fold field to show that this is a fold axial trace**')

    fold_choice=widgets.Text(
        placeholder='Fold',
        value='Fold',
        description='Fold text:',
        disabled=False
    )
    display(fold_choice)

    printmd('**Field that contains information on type of fold (anticline, syncline)**')

    t_choice=widgets.Dropdown(
        options=cols,
        description='Polarity field:',
        disabled=False
    )
    display(t_choice)

    printmd('**Text to search for in field defined by Polarity Field to show that this is a syncline**')

    syn_choice=widgets.Text(
        placeholder='syn',
        value='syn',
        description='Syncline text:',
        disabled=False
    )
    display(syn_choice)


,CATEGORY,FEATURE,FEATURE_D,HWALL_ROSE,geometry
0,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH_EAST,"LINESTRING (415372.000 355736.000, 415462.000 ..."
1,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH,"LINESTRING (419427.000 359339.000, 419546.000 ..."
2,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH_WEST,"LINESTRING (421657.000 359077.000, 421802.000 ..."
3,FAULT,Fault_Inf_HW,"Fault, inferred",NORTH,"LINESTRING (422552.000 370743.000, 422671.000 ..."
4,FAULT,Fault_Inf_HW,"Fault, inferred",EAST,"LINESTRING (409016.000 364784.000, 409039.000 ..."


**Based on file: C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Faults.shp**


**Field that contains information on type of structure (fold, fault etc.)**

Dropdown(description='Fold field:', options=('No_col', 'CATEGORY', 'FEATURE', 'FEATURE_D', 'HWALL_ROSE', 'geom…

**Text to search for in field defined by Fold field to show that this is a fold axial trace**

Text(value='Fold', description='Fold text:', placeholder='Fold')

**Field that contains information on type of fold (anticline, syncline)**

Dropdown(description='Polarity field:', options=('No_col', 'CATEGORY', 'FEATURE', 'FEATURE_D', 'HWALL_ROSE', '…

**Text to search for in field defined by Polarity Field to show that this is a syncline**

Text(value='syn', description='Syncline text:', placeholder='syn')

In [60]:
c_l['ff']=ff_choice.value
c_l['fold']=fold_choice.value
c_l['t']=t_choice.value
c_l['syn']=syn_choice.value

if(not geology_file==''):
    geology=gpd.read_file(geology_file)
    display(geology.head())
    cols=['No_col']
    for sc in geology.columns:
        cols.append(sc)
    printmd('**Based on file:'+' '+geology_file+'**')

    printmd('\n**Field that contains unique ID**')

    o_choice=widgets.Dropdown(
        options=cols,
        description='Unique ID',
        disabled=False
    )
    display(o_choice)

    printmd('**Field that contains finer stratigraphic coding**')

    c_choice=widgets.Dropdown(
        options=cols,
        description='Formation',
        disabled=False
    )
    display(c_choice)

    printmd('\n**Field that contains coarser stratigraphic coding**')

    g_choice=widgets.Dropdown(
        options=cols,
        description='Group',
        disabled=False
    )
    display(g_choice)

    printmd('**Field that contains alternate coarser stratigraphic coding if Group is blank**')

    g2_choice=widgets.Dropdown(
        options=cols,
        description='Group alt',
        disabled=False
    )
    display(g2_choice)


    printmd('**Field that contains alternate stratigraphic coding (used to check for "drift" formations'')**')

    u_choice=widgets.Dropdown(
        options=cols,
        description='Strat alt',
        disabled=False
    )
    display(u_choice)

    printmd('**Field that contains information about lithology**')

    ds_choice=widgets.Dropdown(
        options=cols,
        description='Description:',
        disabled=False
    )
    display(ds_choice)

    printmd('**Field that contains extra lithology information**')

    r1_choice=widgets.Dropdown(
        options=cols,
        description='Litho',
        disabled=False
    )
    display(r1_choice)

    printmd('**Field that contains even more lithology information**')

    r2_choice=widgets.Dropdown(
        options=cols,
        description='Litho alt',
        disabled=False
    )
    display(r2_choice)

    printmd('**Text to search for in field defined by Description code to show that this is a sill**')

    sill_choice=widgets.Text(
        placeholder='sill',
        value='sill',
        description='Sill text',
        disabled=False
    )
    display(sill_choice)

    printmd('**Text to search for in field defined by Litho code to show that this is an intrusion**')

    intrusive_choice=widgets.Text(
        placeholder='intrusive',
        value='intrusive',
        description='Intrusive text',
        disabled=False
    )
    display(intrusive_choice)

    printmd('**Text to search for in field defined by Description code to show that this is an volcanic (not intrusion)**')


    volcanic_choice=widgets.Text(
        placeholder='volc',
        value='volc',
        description='Volc text',
        disabled=False
    )
    display(volcanic_choice)

    printmd('**Field that contains minimum age of unit defined by Formation**')

    min_choice=widgets.Dropdown(
        options=cols,
        description='Min age',
        disabled=False
    )
    display(min_choice)

    printmd('**Field that contains maximum age of unit defined by Formation**')

    max_choice=widgets.Dropdown(
        options=cols,
        description='Max age',
        disabled=False
    )
    display(max_choice)



,OBJECTID,LEX,LEX_D,RANK,BED_EQ,BED_EQ_D,MB_EQ,MB_EQ_D,FM_EQ,FM_EQ_D,GP_EQ,GP_EQ_D,MAX_TIME_Y,MIN_TIME_Y,MAX_AGE,MIN_AGE,MAX_EPOCH,MIN_EPOCH,MAX_SUBPER,MIN_SUBPER,MAX_PERIOD,MIN_PERIOD,LEX_RCS_I,BGSRED,BGSGREEN,BGSBLUE,VERSION,NOM_SCALE,NOM_OS_YR,NOM_BGS_YR,geometry
0,1,ASG,ASHOVER GRIT,BED,ASG,ASHOVER GRIT,NOPAR,NO PARENT,MARSD,MARSDEN FORMATION,MG,MILLSTONE GRIT GROUP [SEE ALSO MIGR],321500000.0,320000000.0,NAMURIAN,NAMURIAN,EARLY PENNSYLVANIAN,EARLY PENNSYLVANIAN,PENNSYLVANIAN,PENNSYLVANIAN,CARBONIFEROUS,CARBONIFEROUS,13211312_ASG-SDST,237,176,0,8.24,50000,1961,1975,"POLYGON ((425512.000 374208.000, 425473.000 37..."
1,2,ASG,ASHOVER GRIT,BED,ASG,ASHOVER GRIT,NOPAR,NO PARENT,MARSD,MARSDEN FORMATION,MG,MILLSTONE GRIT GROUP [SEE ALSO MIGR],321500000.0,320000000.0,NAMURIAN,NAMURIAN,EARLY PENNSYLVANIAN,EARLY PENNSYLVANIAN,PENNSYLVANIAN,PENNSYLVANIAN,CARBONIFEROUS,CARBONIFEROUS,13211312_ASG-SDST,237,176,0,8.24,50000,1975,1978,"MULTIPOLYGON (((421659.000 361494.000, 421643...."
2,3,BES,BLACKSTONE EDGE SANDSTONE,BED,BES,BLACKSTONE EDGE SANDSTONE,NOPAR,NO PARENT,MORRI,MORRIDGE FORMATION,NOPAR,NO PARENT,322000000.0,321500000.0,NAMURIAN,NAMURIAN,EARLY PENNSYLVANIAN,EARLY PENNSYLVANIAN,PENNSYLVANIAN,PENNSYLVANIAN,CARBONIFEROUS,CARBONIFEROUS,13211313_BES-MDSD,201,201,176,8.24,50000,1975,1978,"MULTIPOLYGON (((405636.000 362293.000, 405645...."
3,4,BES,BLACKSTONE EDGE SANDSTONE,BED,BES,BLACKSTONE EDGE SANDSTONE,NOPAR,NO PARENT,MORRI,MORRIDGE FORMATION,NOPAR,NO PARENT,322000000.0,321500000.0,NAMURIAN,NAMURIAN,EARLY PENNSYLVANIAN,EARLY PENNSYLVANIAN,PENNSYLVANIAN,PENNSYLVANIAN,CARBONIFEROUS,CARBONIFEROUS,13211313_BES-SDST,237,176,0,8.24,50000,1975,1978,"MULTIPOLYGON (((407669.000 360721.000, 407636...."
4,5,BLL,BEE LOW LIMESTONE FORMATION,FORMATION,NOTAP,NOT APPLICABLE,NOTAP,NOT APPLICABLE,BLL,BEE LOW LIMESTONE FORMATION,PKLM,PEAK LIMESTONE GROUP,337000000.0,330900000.0,VISEAN,VISEAN,MID MISSISSIPPIAN,MID MISSISSIPPIAN,MISSISSIPPIAN,MISSISSIPPIAN,CARBONIFEROUS,CARBONIFEROUS,13212212_BLL-DOLMST,148,255,148,8.24,50000,1975,1978,"MULTIPOLYGON (((425429.214 355252.195, 425427...."


**Based on file: C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_Bedrock.shp**


**Field that contains unique ID**

Dropdown(description='Unique ID', options=('No_col', 'OBJECTID', 'LEX', 'LEX_D', 'RANK', 'BED_EQ', 'BED_EQ_D',…

**Field that contains finer stratigraphic coding**

...

Text(value='volc', description='Volc text', placeholder='volc')

**Field that contains minimum age of unit defined by Formation**

Dropdown(description='Min age', options=('No_col', 'OBJECTID', 'LEX', 'LEX_D', 'RANK', 'BED_EQ', 'BED_EQ_D', '…

**Field that contains maximum age of unit defined by Formation**

Dropdown(description='Max age', options=('No_col', 'OBJECTID', 'LEX', 'LEX_D', 'RANK', 'BED_EQ', 'BED_EQ_D', '…

In [63]:
c_l['o']=o_choice.value
c_l['g']=g_choice.value
c_l['g2']=g2_choice.value
c_l['c']=c_choice.value
c_l['ds']=ds_choice.value
c_l['u']=u_choice.value
c_l['r1']=r1_choice.value
c_l['r2']=r2_choice.value
c_l['sill']=sill_choice.value
c_l['intrusive']=intrusive_choice.value
c_l['volcanic']=volcanic_choice.value
c_l['min']=min_choice.value
c_l['max']=max_choice.value

mindep=gpd.read_file(mindep_file)
display(mindep.head())
cols=['No_col']
for sc in mindep.columns:
    cols.append(sc)

printmd('**Based on file:'+' '+mindep_file+'**')

printmd('\n**Field that contains site code of deposit**')

msc_choice=widgets.Dropdown(
    options=cols,
    value='SITE_CODE',
    description='Site code',
    disabled=False
)
display(msc_choice)

printmd('**Field that contains short name of deposit (e.g. Jones deposit)**')

msn_choice=widgets.Dropdown(
    options=cols,
    value='SHORT_NAME',
    description='Short Name',
    disabled=False
)
display(msn_choice)

printmd('**Field that contains site type of deposit (Mine, occurrence etc.)**')

mst_choice=widgets.Dropdown(
    options=cols,
    value='SITE_TYPE_',
    description='Site type',
    disabled=False
)
display(mst_choice)


printmd('**Field that contains target commodity of deposit (Commodity)**')

mtc_choice=widgets.Dropdown(
    options=cols,
    value='TARGET_COM',
    description='Target Comm',
    disabled=False
)
display(mtc_choice)

printmd('**Field that contains site commodity of deposit (ore Mineral)**')

mscm_choice=widgets.Dropdown(
    options=cols,
    value='SITE_COMMO',
    description='Site Commodity',
    disabled=False
)
display(mscm_choice)

printmd('**Field that contains commodity group of deposit (e.g. Metal)**')

mcom_choice=widgets.Dropdown(
    options=cols,
    value='COMMODITY_',
    description='Commodity',
    disabled=False,
    style=style
)
display(mcom_choice)

printmd('**Text to search for in field defined by Site type code that shows site to ignore**')

minf_choice=widgets.Text(
    placeholder='Cons',
    value='Cons',
    description='Not mine text',
    disabled=False
)
display(minf_choice)


,SITE_CODE,SITE_TITLE,SHORT_NAME,SITE_COMMO,SITE_TYPE_,SITE_SUB_T,SITE_STAGE,TARGET_COM,COMMODITY_,PROJ_CODE,PROJ_TITLE,WEB_LINK,EXTRACT_DA,geometry
0,S0001482,Vivash Gorge,Vivash Gorge,Fe,Deposit,Unspecified,Undeveloped,IRON ORE,IRON,J03931,Brockman 4 Area / Rio Tinto,http://minedexext.dmp.wa.gov.au/minedex/extern...,2018-05-07,POINT (505671.537 7495706.968)
1,S0231563,Eliwana Exploration Camp,Exploration camp,Fe,Infrastructure,Town/Village/Camp,Operating,IRON ORE,IRON,J03436,Western Hub / FMG,http://minedexext.dmp.wa.gov.au/minedex/extern...,2018-05-07,POINT (503344.199 7511681.103)
2,S0233921,Eliwana Far East 2 Marra Mamba,Eliwana Far East 2 Marra Mamba,Fe,Deposit,Unspecified,Undeveloped,IRON ORE,IRON,J03436,Western Hub / FMG,http://minedexext.dmp.wa.gov.au/minedex/extern...,2018-05-07,POINT (505699.008 7509917.737)


**Based on file: C:/Users/00073294/Dropbox/1_Jupyter_notebooks/Peak_District/null_mindeps.shp**


**Field that contains site code of deposit**

Dropdown(description='Site code', index=1, options=('No_col', 'SITE_CODE', 'SITE_TITLE', 'SHORT_NAME', 'SITE_C…

**Field that contains short name of deposit (e.g. Jones deposit)**

...

Dropdown(description='Site Commodity', index=4, options=('No_col', 'SITE_CODE', 'SITE_TITLE', 'SHORT_NAME', 'S…

**Field that contains commodity group of deposit (e.g. Metal)**

Dropdown(description='Commodity', index=9, options=('No_col', 'SITE_CODE', 'SITE_TITLE', 'SHORT_NAME', 'SITE_C…

**Text to search for in field defined by Site type code that shows site to ignore**

Text(value='Cons', description='Not mine text', placeholder='Cons')

In [64]:
c_l['msc']=msc_choice.value
c_l['msn']=msn_choice.value
c_l['mst']=mst_choice.value
c_l['mtc']=mtc_choice.value
c_l['mscm']=mscm_choice.value
c_l['mcom']=mcom_choice.value
c_l['minf']=minf_choice.value

if(not os.path.isdir(test_data_name)):
    os.mkdir(test_data_name)
f=open(test_data_name+'/'+test_data_name+'_config.hjson',"w")
f.write(str(c_l).replace('\n',''))
f.write("\n")

f.close()

In [65]:
c_l

{'d': 'DIP',
 'dd': 'AZIMUTH',
 'sf': 'NEWBGSVS2',
 'bedding': 'rata',
 'otype': 'dip direction',
 'bo': 'FEATURE',
 'btype': 'Blah',
 'g': 'GP_EQ_D',
 'g2': 'MAX_SUBPER',
 'c': 'FM_EQ_D',
 'ds': 'LEX_D',
 'u': 'MIN_SUBPER',
 'r1': 'LEX',
 'r2': 'GP_EQ',
 'sill': 'sill',
 'intrusive': 'INTRUSION',
 'volcanic': 'volc',
 'msc': 'SITE_CODE',
 'msn': 'SHORT_NAME',
 'mst': 'SITE_TYPE_',
 'mtc': 'TARGET_COM',
 'mscm': 'SITE_COMMO',
 'mcom': 'COMMODITY_',
 'minf': 'Cons',
 'min': 'MIN_TIME_Y',
 'max': 'MAX_TIME_Y',
 'f': 'FEATURE',
 'fault': 'Fault',
 'ff': 'FEATURE',
 'fold': 'Fold',
 'fdip': 'No_col',
 'fdipnull': '0',
 'fdipdir': 'No_col',
 'fdipdir_flag': 'alpha',
 'fdipest': 'No_col',
 'fdipest_vals': 'No_col',
 'n': 'No_col',
 't': 'HWALL_ROSE',
 'syn': 'syn',
 'o': 'OBJECTID',
 'gi': 'OBJECTID',
 'deposit_dist': 500}

In [66]:
f=open(test_data_name+'/run_m2l.py',"w")
f.write("\
\
import os\n\
from map2loop.project import Project\n\
\n\
proj = Project(\n\
                  geology_file='"+geology_file+"',\n\
                  fault_file='"+fault_file+"',\n\
                  fold_file='"+fold_file+"',\n\
                  structure_file='"+structure_file+"',\n\
                  mindep_file='"+mindep_file+"',\n\
                  metadata='"+"./"+test_data_name+"_config.hjson'\n\
                )\n\
\n\
proj.update_config(\n\
                    out_dir='"+test_data_name+"',\n\
                    overwrite='true',\n\
                    bbox_3d={\n\
                         'minx': "+minx+" ,\n\
                         'miny': "+miny+" ,\n\
                         'maxx': "+maxx+" ,\n\
                         'maxy': "+maxy+" ,\n\
                         'base': "+minz+",\n\
                         'top':  "+maxz+",\n\
                         'local' : True,\n\
                     },\n\
                    proj_crs={'init': '"+dst_crs+"'},\n\
                    quiet=True\n\
                  )\n\
\nproj.run(aus=False)\n"   
        
)
f.write("\n")
f.close()